In [1]:
import Simulation
import time 
import pandas as pd 
import numpy as np
import copy
import concurrent.futures
import pickle
import random

# Simulation Functions

In [2]:
""" Run multiple simulations
Parameters
----------
test_set: dict
    Contains all jobs scheduled on all levels. Structure of the dict:Level:dict:Machines:Df
n: int
    number of simulations

Returns
-------
DF
    all simulation results in one dataframe: columns: level, start_planned, end_planned, end_new, disruption_buffer
    
"""
def multiple_simulations(test_set, n): 
    results = []

    l = len(test_set)

    # Get random seeds to guarantee randomness in parallelisation 
    random_seed_array = [list(np.random.randint(0, 2**32 - 1,l))]
    for i in range (1,n): 
        random_seed_array.append(list(np.random.randint(0, 2**32 - 1,4)))
        
        
    with concurrent.futures.ProcessPoolExecutor() as executor: 
        for i in range(n):
            results.append(executor.submit(Simulation.simulation, test_set, random_seed_array[i]))

    # Processing the output of the parallel simulations
    end_times_final = pd.DataFrame(columns=['level', 'start_planned', 'end_planned', 'end_new',  'disruption_buffer'])

    for f in concurrent.futures.as_completed(results):
        for index_level, dict_level in f.result().items():
            for machine_index, machine_df in dict_level.items(): 
                end_times_temp = pd.DataFrame(columns=['level', 'start_planned', 'end_planned', 'end_new', 'disruption_buffer'])
                end_times_temp['end_planned'] = machine_df['end_planned'].copy()
                end_times_temp['end_new'] = machine_df['end_new'].copy()
                end_times_temp['end_new'] = pd.to_datetime(end_times_temp['end_new'])
                end_times_temp['level'] = index_level
                end_times_temp['start_planned'] = machine_df['start_planned'].copy()
                end_times_temp['disruption_buffer'] = machine_df['disruption_buffer'].copy()
                end_times_final = pd.concat([end_times_final, end_times_temp])

    
    return end_times_final

In [3]:
# open testset
test_set = pickle.load(open('example_orders_anonymous.pickle', 'rb'))

### Test: Run 5 simulations

In [4]:
multiple_simulations(test_set,5)

,level,start_planned,end_planned,end_new,disruption_buffer
1,1,2020-01-16 00:00:00.000000000,2020-01-17 08:36:41.715500032,2020-01-17 18:07:06.590350000,00:10:00
28,1,2020-01-17 08:36:41.715500032,2020-01-19 10:46:29.200450048,2020-01-20 08:21:45.094200032,00:06:00
54,1,2020-01-19 10:46:29.200450048,2020-01-24 12:46:27.854800128,2020-01-26 19:42:32.201150048,00:25:00
95,1,2020-01-24 12:46:27.854800128,2020-01-26 12:37:13.961650176,2020-01-27 07:49:51.854250128,00:15:00
111,1,2020-01-26 12:37:13.961650176,2020-01-28 13:09:39.873150208,2020-01-29 05:32:29.718900176,00:06:00
...,...,...,...,...,...
176,4,2020-02-29 15:49:28.188750336,2020-03-03 21:05:57.585000448,2020-03-09 00:15:46.156850000,00:15:00
210,4,2020-03-06 16:06:13.041000192,2020-03-08 13:28:17.388000256,2020-03-11 04:01:05.628149999,00:25:00
227,4,2020-03-13 10:07:51.533099776,2020-03-16 22:23:07.261799680,2020-03-22 13:47:48.122350064,00:25:00
248,4,2020-03-21 23:38:06.825299968,2020-03-27 04:18:22.408499968,2020-04-05 01:37:00.217749968,00:10:00
